In [2]:
import numpy as np
from dm_control import mujoco
import mediapy as media
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'dm_control'

In [ ]:
physics = mujoco.Physics.from_xml_path("../anybotics_anymal_c/anymal_c.xml")
nv = physics.model.nv
nu = physics.model.nu
pixels = physics.render()
media.show_image(pixels)